<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E9%80%B1_%E7%B0%A1%E5%96%AE%E7%9A%84%E7%B7%9A%E6%80%A7%E8%BF%B4%E6%AD%B8%E6%A8%A1%E5%9E%8B_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 讀取Excel檔案

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 讀取 Excel 文件
file_path = 'Price.xls'  # 替換為 Excel 文件的路徑
df = pd.read_excel(file_path, skiprows=1)

# 查看數據框中是否有缺失值
#print(df.isnull().sum())

# 選擇適合的處理方式，這裡選擇直接刪除包含 NaN 的行
df = df.dropna()

# 把樓別和樓高(這棟共有幾樓)分離出來


In [ ]:
import re

df['樓別/樓高'] = df['樓別/樓高'].str.replace('夾層', '')

# 假設 df 是你的數據框，首先定義一個函數來處理不同的樓別/樓高格式
def extract_levels(row):
    # 使用正則表達式提取層數，支持數字和中文數字，並忽略多餘的部分
    match = re.match(r'(\D*)(\d+|\w+)層.*?(\d+|\w+)層', row)
    #print(match)
    if match:
        lower_level = match.group(2)  # 樓別（lower level）
        upper_level = match.group(3)  # 樓高（upper level）
        #print(lower_level, upper_level)
        return pd.Series([lower_level, upper_level])
    else:
        return pd.Series([None, None])  # 如果無法匹配則返回 None

# 應用提取函數，添加新列
df[['樓別', '樓高']] = df['樓別/樓高'].apply(extract_levels)

# 轉換為數值類型，處理中文數字
def chinese_to_number(chinese_str):
    chinese_numerals = {
        '零': 0, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
        '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
    }
    number = 0
    if '十' in chinese_str and len(chinese_str)>1:
        #print(chinese_str)
        parts = chinese_str.split('十')
        number += chinese_numerals[parts[0]] * 10 if parts[0] else 10
        if len(parts) > 1:
            #print(parts)
            number += chinese_numerals[parts[1]]
    else:
        number = chinese_numerals.get(chinese_str, None)
    return number

# 將樓別和樓高轉換為數值
df['樓別'] = df['樓別'].apply(chinese_to_number)
df['樓高'] = df['樓高'].apply(chinese_to_number)

# 確保類型為浮點數
df['樓別'] = df['樓別'].astype(float)
df['樓高'] = df['樓高'].astype(float)

# 其它資料清理與型態轉換

In [ ]:

# 去掉百分號並將"主建物佔比"轉換為浮點數
df['主建物佔比'] = df['主建物佔比'].str.replace('%', '').astype(float) / 100

# 選擇特徵和目標變量
# 需要選擇適合的特徵列，並進行適當的數據清理，例如將「總價(萬元)」和「單價(萬元/坪)」的數值去掉逗號
# 確保這些列是字符串類型，以便使用 .str.replace
df['總價(萬元)'] = df['總價(萬元)'].astype(str).str.replace(',', '').astype(float)
df['單價(萬元/坪)'] = df['單價(萬元/坪)'].astype(str).str.replace(',', '').astype(float)

# 對所有可能為數值的列進行相同的清理操作
def clean_column(column):
    return column.str.replace(',', '').astype(float)

# 如果列中有其他需要處理的數值列，可以在這裡添加
df['車位總價(萬元)'] = clean_column(df['車位總價(萬元)'].astype(str))

# 將'有無電梯'轉換為二進制數值（0表示沒有電梯，1表示有電梯）
df['有無電梯'] = df['電梯'].apply(lambda x: 1 if x == '有' else 0)
df['有無管理組織'] = df['管理組織'].apply(lambda x: 1 if x == '有' else 0)
print(df.head(5))

      編號            地段位置或門牌  社區簡稱       交易日期  總價(萬元)  單價(萬元/坪)  總面積(坪)  \
76    77   大安區大安路二段３巷５３號十二樓  忠泰鳳磐  113/08/11  8000.0     105.3   88.71   
78    79  大安區新生南路一段９７巷１號十二樓   在壹起  113/08/09  8400.0     132.7   77.75   
81    82   大安區新生南路一段９７巷３號六樓   在壹起  113/08/09  7358.0     128.5   64.36   
114  115  大安區嘉興街３２３巷３號十四樓之２   和平賞  113/07/13  5313.0      91.2   58.28   
161  162     大安區安和路一段１１３號十樓  國揚安和  113/07/10  3819.0     100.0   43.32   

      主建物佔比               型態    屋齡  ...          交易筆棟數  建物現況格局 車位總價(萬元) 管理組織  \
76   0.5835  住宅大樓(11層含以上有電梯)   7.0  ...  土:1 建物:1 車位:2    2廳2衛    700.0    有   
78   0.5949  住宅大樓(11層含以上有電梯)   1.0  ...  土:1 建物:1 車位:2  3房2廳2衛    670.0    有   
81   0.5741  住宅大樓(11層含以上有電梯)   1.0  ...  土:1 建物:1 車位:1  3房2廳2衛    342.0    有   
114  0.5411  住宅大樓(11層含以上有電梯)  24.0  ...  土:2 建物:1 車位:1  3房2廳2衛    300.0    有   
161  0.9839  住宅大樓(11層含以上有電梯)  45.0  ...  土:1 建物:1 車位:1  4房2廳3衛    250.0    有   

    電梯                                    備註    樓別    樓高 有無電梯  有無管理組織  
76

# 基本多元線性回歸範例

In [ ]:


# 選擇相關列作為特徵和目標
X = df[['單價(萬元/坪)', '總面積(坪)', '主建物佔比', '屋齡', '樓別', '樓高', '車位總價(萬元)', '有無電梯', '有無管理組織']]
y = df['總價(萬元)']

# 將數據分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 創建並訓練線性回歸模型
model = LinearRegression()
model.fit(X_train, y_train)

# 進行預測
y_pred = model.predict(X_test)

# 輸出結果
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R2):", r2_score(y_test, y_pred))
print("Model Coefficients:", model.coef_)
print("Intercept:", model.intercept_)


Mean Squared Error (MSE): 1483596.424745998
R-squared (R2): 0.6563539358457346
Model Coefficients: [   64.01447299   114.947535   -2036.93898469    46.56603814
   -95.67547025    54.59873967    -2.20479702     0.
     0.        ]
Intercept: -6846.392048947228


# 進階一點的方式

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 數據預處理
X = df[['單價(萬元/坪)', '總面積(坪)', '主建物佔比', '屋齡', '樓別', '樓高', '車位總價(萬元)', '有無電梯', '有無管理組織']]
y = df['總價(萬元)']

# 填補缺失值
X.fillna(X.mean(), inplace=True)

# 特徵縮放
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 數據分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 超參數調優示例：使用嶺回歸
param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}  # 可以調整的參數
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)  # 打印最佳參數
print(grid_search.best_score_)  # 打印最佳模型的得分

# 獲取最佳模型
best_model = grid_search.best_estimator_

# 進行預測
y_pred = best_model.predict(X_test)

# 輸出結果
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R2):", r2_score(y_test, y_pred))
print("Best Model Coefficients:", best_model.coef_)
print("Intercept:", best_model.intercept_)


{'alpha': 0.1}
-699426.7319480918
Mean Squared Error (MSE): 1197396.2510836998
R-squared (R2): 0.7226466025028105
Best Model Coefficients: [ 897.99568091 2057.07275533 -221.35088302  412.79856031 -207.63010903
  352.9317023   -76.88173151    0.            0.        ]
Intercept: 5399.846724827197


<ipython-input-11-cf6fe0a6ff6c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


# 特徵影響力解釋

In [ ]:
import pandas as pd
from sklearn.linear_model import Ridge

# 假設你已經準備好X和y
model = Ridge(alpha=1.0)
model.fit(X, y)

# 獲取特徵係數
coef = pd.Series(model.coef_, index=X.columns)
print(coef.sort_values(ascending=False))  # 按照影響力從大到小排序


總面積(坪)      111.653307
單價(萬元/坪)     40.233638
樓高           15.232749
屋齡            8.106793
樓別            0.334471
有無電梯          0.000000
有無管理組織        0.000000
車位總價(萬元)     -1.773624
主建物佔比      -338.951324
dtype: float64


# 預測其他的欄位，如單價

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 數據預處理
X = df[[ '總面積(坪)', '主建物佔比', '屋齡', '樓別', '樓高', '車位總價(萬元)']]
y = df['單價(萬元/坪)']

# 填補缺失值
X.fillna(X.mean(), inplace=True)

# 特徵縮放
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 數據分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 超參數調優示例：使用嶺回歸
param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}  # 可以調整的參數
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 獲取最佳模型
best_model = grid_search.best_estimator_

# 進行預測
y_pred = best_model.predict(X_test)

# 輸出結果
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R2):", r2_score(y_test, y_pred))
print("Best Model Coefficients:", best_model.coef_)
print("Intercept:", best_model.intercept_)



<ipython-input-10-8d69be3dc0f5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


Mean Squared Error (MSE): 578.349656777621
R-squared (R2): -0.9206050236881171
Best Model Coefficients: [-8.12905749 -0.1735921  -9.21064662  6.9467291  -1.8265459  12.03782992]
Intercept: 119.34316017290922


# 基本的多元線性迴歸 函式模版

In [ ]:
#----------- model.py -----------------------------

# 若是做為一支單獨的 model.py 該 import 的模組都要放進來
# from sklearn.linear_model import LinearRegression
# ...

def Select_feature(df):
  # 選擇相關列作為特徵和目標
  X = df[[ '車位總價(萬元)', '有無電梯', '有無管理組織']]
  return X

def Training(X_train, y_train):
  # 創建並訓練線性回歸模型
  model = LinearRegression()
  model.fit(X_train, y_train)
  return model


#----------- main.py -----------------------------

X = Select_feature(df) #這裡呼叫同學們寫的選擇特徵函式，並回傳篩選好的特徵
y = df['總價(萬元)']

# 將數據分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = Training(X_train, y_train) #這裡呼叫同學們寫的訓練函式，並回傳訓練好的模型

# 進行預測
y_pred = model.predict(X_test)

# 輸出結果
# print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R2):", r2_score(y_test, y_pred))
# print("Model Coefficients:", model.coef_)
# print("Intercept:", model.intercept_)

r2 = r2_score(y_test, y_pred)
threshold = float(input())
print(r2>threshold)


R-squared (R2): 0.41870819699163886
04
False


# 進階版的多元線性迴歸作業模版

In [ ]:
#----------- model.py -----------------------------
# 若是做為一支單獨的 model.py 該import 的模組都要放進來
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import Ridge
# from sklearn.preprocessing import StandardScaler
# ...

def Select_feature(df):
  # 選擇相關列作為特徵和目標
  X = df[['車位總價(萬元)', '有無電梯', '有無管理組織']]
  # 填補缺失值
  X.fillna(X.mean(), inplace=True)

  # 特徵縮放
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  return X_scaled

def Training(X_train, y_train):
  # 超參數調優示例：使用嶺回歸
  param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}  # 可以調整的參數
  ridge = Ridge()
  grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
  grid_search.fit(X_train, y_train)

  # 獲取最佳模型
  best_model = grid_search.best_estimator_

  return best_model

#----------- main.py -----------------------------

#X = df[['單價(萬元/坪)', '總面積(坪)', '主建物佔比', '屋齡', '樓別', '樓高', '車位總價(萬元)', '有無電梯', '有無管理組織']]

X = Select_feature(df) #這裡呼叫同學們寫的選擇特徵函式，並回傳篩選好的特徵
y = df['總價(萬元)']

# 將數據分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = Training(X_train, y_train) #這裡呼叫同學們寫的訓練函式，並回傳訓練好的模型

# 進行預測
y_pred = model.predict(X_test)

# 輸出結果
# print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R2):", r2_score(y_test, y_pred))
# print("Model Coefficients:", model.coef_)
# print("Intercept:", model.intercept_)

r2 = r2_score(y_test, y_pred)
threshold = float(input())
print(r2>threshold)


<ipython-input-12-2a4d7a858208>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


R-squared (R2): 0.4322105780235991
1
False
